In [39]:
%%capture
%%bash

## Loading Enviromet

## packages installation for running notebook
pip install nglview
pip install ipywidgets==7.0.0
# pip install biopython
pip install rdkit-pypi

# pip install "biobb_io==3.8.0"
# pip install "biobb_structure_utils>=3.8.0"
# pip install "biobb_vs>=3.8.1"

In [40]:
from google.colab import output
output.enable_custom_widget_manager()

In [41]:
# import drive (it may ask for permission)
from google.colab import drive
drive.mount('/content/drive')

PATHWDIR = '/content/drive/MyDrive/MolDock4Colab'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import os
### INSTAL SMINA !!!
SMINA = os.path.join(PATHWDIR, 'smina.static')
SMINA

! wget https://sourceforge.net/projects/smina/files/smina.static/download -O '{SMINA}'
! chmod u+x '{SMINA}'

--2023-04-19 14:18:05--  https://sourceforge.net/projects/smina/files/smina.static/download
Resolving sourceforge.net (sourceforge.net)... 104.18.11.128, 104.18.10.128, 2606:4700::6812:a80, ...
Connecting to sourceforge.net (sourceforge.net)|104.18.11.128|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.sourceforge.net/project/smina/smina.static?ts=gAAAAABkP_geqNeQVVVZHK76N3H3yEkyhg5uP-_lgaiO1ByF4oWyBKBt_eSupFs2JlFGbl-pkmoPzD1eaXmVFwO3sneN28n6hQ%3D%3D&use_mirror=cfhcable&r= [following]
--2023-04-19 14:18:06--  https://downloads.sourceforge.net/project/smina/smina.static?ts=gAAAAABkP_geqNeQVVVZHK76N3H3yEkyhg5uP-_lgaiO1ByF4oWyBKBt_eSupFs2JlFGbl-pkmoPzD1eaXmVFwO3sneN28n6hQ%3D%3D&use_mirror=cfhcable&r=
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Connecting to downloads.sourceforge.net (downloads.sourceforge.net)|204.68.111.105|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: 

# 3. Running Molecular Docking as in a Virtual Screening Experiment

Virtual Screening (VS) methods are utilized to sift through vast databases of compounds and identify promising drug candidates or biologically active molecules through computer-based screening. One of the most popular techniques employed in this regard is Molecular Docking, which simulates the interactions between molecules to predict their binding affinities and determine their suitability for therapeutic use


You can take a look to [this article](https://www.sciencedirect.com/science/article/pii/B9780128191323000142?via%3Dihub) were some basics of VS are prsented.

For running this notebook we need the files obtained in previous steps: receptor in pdbqt, box.pdb, and chemical library created with Data Warrior.
This notebook uses SMINA. 

**Outline:**
    
    3.1. Load Target
    3.2. Load Chemical Library
    3.3. Perform VS with SMINA
    


About SMINA:

SMINA is a fork of Autodock Vina (http://vina.scripps.edu/) that 
focuses on improving scoring and minimization.  Changes from the
standard Vina (version 1.1.2) include:

 - **comprehensive support for ligand molecular formats (via OpenBabel)**
 -support for multi-ligand files (e.g., an sdf file)*
 -support for addition term types (e.g., desolvation, electrostatics)
 -support for custom, user-parameterized scoring functions (see --custom_scoring)
 -automatic box creation based on a user-specified bound ligand
 -allow the output of more than 20 docking poses
 -vastly improved minimization algorithms (--minimize goes to convergence)
 
For workflows where AutoDock Vina is used for minimization (local_only) 
as opposed to of docking, these changes make Vina much easer to use and 
10-20x faster. Docking performance is about the same since partial charge 
calculation and file i/o isn't such a big part of the performance.

If you find smina useful, please cite our paper: 
http://pubs.acs.org/doi/abs/10.1021/ci300604z

**Non-pdbqt ligand files must have partial charges added.  This is done
using OpenBabel and will get different results than the prepare_ligand4.py
script that comes with AutoDock Tools.**


Pre-built binaries are provided that were built on Ubuntu 14.04.  The main
dependencies are boost (1.54) and openbabel.  A static binary is provided
in case these dependencies cannot be met (however, it probably still will
not work if the kernel is older than 2.6.24).

```
Input:
  -r [ --receptor ] arg rigid part of the receptor (***PDBQT***)

  -l [ --ligand ] arg   ligand(s)


Search space (required):
  --center_x arg        X coordinate of the center
  --center_y arg        Y coordinate of the center
  --center_z arg        Z coordinate of the center
  --size_x arg          size in the X dimension (Angstroms)
  --size_y arg          size in the Y dimension (Angstroms)
  --size_z arg          size in the Z dimension (Angstroms)
  
  or
  
  --autobox_ligand arg  Ligand to use for autobox


Scoring and minimization options:

  --score_only                 score provided ligand pose


Output (optional):
  -o [ --out ] arg      output file name, format taken from file extension
  --log arg             optionally, write log file


Misc (optional):
  --cpu arg                  the number of CPUs to use (the default is to try 
                             to detect the number of CPUs or, failing that, use
                             1)
  --seed arg                 explicit random seed
  --exhaustiveness arg (=8)  exhaustiveness of the global search (roughly 
                             proportional to time)
  --num_modes arg (=9)       maximum number of binding modes to generate
  --energy_range arg (=3)    maximum energy difference between the best binding
                             mode and the worst one displayed (kcal/mol)
  --min_rmsd_filter arg (=1) rmsd value used to filter final poses to remove 
                             redundancy
  -q [ --quiet ]             Suppress output messages
  --addH arg                 automatically add hydrogens in ligands (on by 
                             default)


Configuration file (optional):
  --config arg          the above options can be put here
  ```
  



## 3.1. Load Target

In [43]:
import os
import re

PATH = os.path.join(PATHWDIR, 'Output') # your output path

if not os.path.exists(PATH):
   os.makedirs(PATH)

pdb_code = "2BRC"         # 2BRC + Hsp90 Inhibitor
ligand_code = "CT5"       # Hsp90 Inhibitor

## Load
ProteinForDocking = os.path.join(PATH, 'prep_receptor.pdbqt') # Protein in pdbqt
LigandFromProtein = os.path.join(PATH, 'pdb_ligand.pdb') # Crystalized ligand
output_box = os.path.join(PATH, "box.pdb") # box size

## Extract box information from pdb file.
with open(output_box) as file:
    lines = file.readlines()
    
pattern = '[-+]?\d+\.\d+'
box_data = re.findall(pattern, lines[0])
print(box_data)
x, y, z, x_size, y_size, z_size = box_data

['19.365', '-3.578', '-4.566', '11.884', '12.914', '13.647']


## 3.2.  Load Chemical Library

Copy to your folder your chemical library built in Data Warrior.


In [44]:
# You may need to change the file's name to coincide with yours. 
ConfoutputFilePath = os.path.join(PATH, 'DockingCompoundsPubChem.sdf')

## 3.3. Perform VS with SMINA

In [ ]:
# You can uncomment this to make sure that your SMINA will execute 
# ! '{SMINA}' --help # test

### 3.3.1. Score only !

In molecular docking, scoring is the process of evaluating how well a ligand (small molecule) fits into a binding site on a protein or other macromolecule. Scoring functions are used to predict the binding affinity between the ligand and the target molecule, which can help to identify promising lead compounds for drug development.

AutoDock Vina is a popular software tool for molecular docking that uses a scoring function to rank the predicted binding poses of ligands in a binding site. The "score only" option in Vina allows users to quickly evaluate the binding affinity of a set of ligands without performing a full docking simulation. Instead, the ligands are placed in the binding site and scored based on their predicted binding affinity, using a simplified version of the Vina scoring function.


In [45]:
DockedScoreOnly = os.path.join(PATH, 'DockedScoreOnly.pdb')
LogScoreOnly = os.path.join(PATH, 'OutputScoreonly.txt')
# --score_only   

In [46]:
! '{SMINA}' --seed 0  -r '{ProteinForDocking}' -l '{LigandFromProtein}' -o '{DockedScoreOnly}' --log '{LogScoreOnly}' --score_only 

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

## Name gauss(o=0,_w=0.5,_c=8) gauss(o=3,_w=2,_c=8) repulsion(o=0,_c=8) hydrophobic(g=0.5,_b=1.5,_c=8) non_dir_h_bond(g=-0.7,_b=0,_c=8) num_tors_div
Affinity: -8.13113 (kcal/mol)
Intramolecular energy: -0.90629
Term values, before weighting:
##  78.34360 1374.94447 2.84795 29.69333 1.75715 0.00000
Refine time 0.

For this the score only is:

In [48]:
# Extract affinity
with open(LogScoreOnly) as file:
    lines = file.readlines()
#lines
pattern = '(?<=Affinity: )[-+]?\d+\.\d+'
list_app = [re.findall(pattern, line) for line in lines]

affinity_so = [value for value in list_app if value][0][0]
print(f'Affinity in score only: {affinity_so} kcal/mol')

Affinity in score only: -8.13113 kcal/mol


### 3.3.2. VS with SMINA

Now we will execute several simulations (as much as ligands you have in the chemical library)

In [49]:
DockedFilePath = os.path.join(PATH, 'All_Docked.sdf') # try with uncompressed file !!
LogOuputDockingVS = os.path.join(PATH, 'OutputDockingVS.txt')
ExhaustivenessValue = 8
nmodes = 8

Our ligand is now the Confoutut File path (our sdd file)

In [50]:
# the \ is only for splitting the line and read it better
# random seed can be removed !

! '{SMINA}' --seed 0 \
--center_x '{x}' --center_y '{y}' --center_z '{y}' \
--size_x '{x_size}' --size_y '{y_size}' --size_z '{z_size}' \
-r '{ProteinForDocking}' -l '{ConfoutputFilePath}' \
--num_modes '{nmodes}' \
-o '{DockedFilePath}' --exhaustiveness '{ExhaustivenessValue}' --log '{LogOuputDockingVS}'


   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 0

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
1       -

We also have the results in  the output file.

In [51]:
from rdkit.Chem import PandasTools
from rdkit import RDLogger

def disable_rdkit_logging():
    """
    Disables RDKit whiny logging.
    """
    import rdkit.rdBase as rkrb
    import rdkit.RDLogger as rkl
    logger = rkl.logger()
    logger.setLevel(rkl.ERROR)
    rkrb.DisableLog('rdApp.error')
    

disable_rdkit_logging()

DockedFilePath
FILE_TEST = os.path.join(PATH, 'All_Docked.sdf')

df_pandas = PandasTools.LoadSDF(FILE_TEST,molColName='Molecule')
df_pandas[:3]

,minimizedAffinity,ID,Molecule
0,-6.26799,129716650,<rdkit.Chem.rdchem.Mol object at 0x7f876ca66510>
1,-6.21852,129716650,<rdkit.Chem.rdchem.Mol object at 0x7f876ca66dd0>
2,-6.10948,129716650,<rdkit.Chem.rdchem.Mol object at 0x7f874c732f90>


In [58]:
DockedFilePath

'/content/drive/MyDrive/MolDock4Colab/Output/All_Docked.sdf'

### 3.3.3. Visualization

In [59]:
import ipywidgets

exs_models = range(1, len(df_pandas.ID.unique())+1)
exs_models
assert len(exs_models) == len(df_pandas.ID.unique()) , 'shape problem'

exs_nmodes = range(1,nmodes+1)

assert len(exs_nmodes) == nmodes, 'shape problem 2'

mdsel_molecule = ipywidgets.Dropdown(
    options=exs_models,
    description='Sel. molecule:',
    disabled=False,
)


mdsel_mode = ipywidgets.Dropdown(
    options=exs_nmodes,
    description='Sel. mode:',
    disabled=False,
)

display(mdsel_molecule)

display(mdsel_mode)

# call as  mdsel_molecule.value, mdsel_mode.value

A Jupyter Widget

A Jupyter Widget

In [60]:
nmolecule =  mdsel_molecule.value #3
nmodel = mdsel_mode.value 
assert nmodel <= nmodes, 'over selecting'

models = f'/{nmodel + (nmolecule-1)*nmodes}'

print(f'Molecule: {nmolecule}, nmodel: {nmodel}, Model {models[1:]}')


Molecule: 1, nmodel: 1, Model 1


DockedFilePath

In [61]:
#models = 'all'
# un compres file for execution
FILE_TEST = os.path.join(PATH, 'All_Docked.sdf')
FILE_TEST # shoiuld be uncompressed DockedFilePath

'/content/drive/MyDrive/MolDock4Colab/Output/All_Docked.sdf'

In [62]:
import nglview as nv
from ipywidgets import HBox
from google.colab import output
output.enable_custom_widget_manager()

#models = 'all'
models = f'/{nmodel + (nmolecule-1)*nmodes}'
print(f'Molecule: {nmolecule}, nmodel: {nmodel}, Model {models[1:]}')

v0 = nv.show_structure_file(FILE_TEST, default=False)
v0.add_representation(repr_type='licorice', 
                        selection=models,
                       colorScheme= 'partialCharge')
v0.center()
v1 = nv.show_structure_file(FILE_TEST, default=False)
v1.add_representation(repr_type='ball+stick', 
                        selection=models)
v1.center()

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

Molecule: 1, nmodel: 1, Model 1


A Jupyter Widget

Let us see it all together, the protein structure and the docked ligand. 

In [64]:
print(f'Molecule: {nmolecule}, nmodel: {nmodel}, Model {models[1:]}')
print(df_pandas.iloc[int(models[1:])-1]) # as model correspond to row index -1 because starting value


import numpy as np
bindingsite_data = os.path.join(PATH, 'bindingsite.npy')
bindingSite = np.load(bindingsite_data)


view = nv.NGLWidget()

# v1 = Experimental Structure
v1 = view.add_component(ProteinForDocking)

v1.add_representation(repr_type='licorice', 
                     selection='STI',
                     radius=0.5)

# v2 = Docking result
v2 = view.add_component(FILE_TEST, default=False)
v2.add_representation(repr_type='ball+stick', colorScheme='element', #color= 'green', 
                        selection=models)
v2.center()

# v3 = surface
#view.add_representation(repr_type='surface', 
#                        selection=', '.join(str(v) for v in bindingSite), 
#                        color='pink',
#                        lowResolution= True,
#                        smooth=0.4)

view._remote_call('setSize', target='Widget', args=['','500px'])

Molecule: 1, nmodel: 1, Model 1
minimizedAffinity                                            -6.26799
ID                                                          129716650
Molecule             <rdkit.Chem.rdchem.Mol object at 0x7f876ca66510>
Name: 0, dtype: object


In [72]:
view

A Jupyter Widget

If the visualization is not working in your google colab, please visualize it in Pymol in your personal laptop.
You have to load the receptor, the score only molecule (i.e, the cristalized ligand) and all the poses from your candidates.

```
load prep_receptor.pdbqt
load All_Docked.sdf
load DockedScoreOnly.pdb
```

Later try to answer... binds any of your molecules better than the known ligand?

